In [39]:
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
min_val = 0
max_val = 100

opcash_to_capex = pd.read_csv('opcash_to_capex_tech.csv').drop(columns=['Unnamed: 0'])
opcash_to_capex = opcash_to_capex[(opcash_to_capex['Operating cash flow to capex'] <= max_val)&
                                  (opcash_to_capex['Operating cash flow to capex'] >= min_val) &
                                  ~(opcash_to_capex['capex'] == 0)]
opcash_to_capex.sort_values(by='Operating cash flow to capex', ascending=True)
annual_returns = pd.read_csv('annual_returns.csv').rename(columns={'Date':'date', 'Close': 'close(%)'})
daily_returns = pd.read_csv('daily_close.csv').rename(columns={'Date':'date', 'Close': 'close'})
daily_returns

,date,close,ticker
0,1994-01-10 00:00:00-05:00,0.248125,AAPL
1,1994-01-11 00:00:00-05:00,0.235212,AAPL
2,1994-01-12 00:00:00-05:00,0.225065,AAPL
3,1994-01-13 00:00:00-05:00,0.225988,AAPL
4,1994-01-14 00:00:00-05:00,0.228755,AAPL
...,...,...,...
1315928,2024-01-03 00:00:00-05:00,10.840000,WAVS
1315929,2024-01-04 00:00:00-05:00,10.860000,WAVS
1315930,2024-01-05 00:00:00-05:00,10.890000,WAVS
1315931,2024-01-08 00:00:00-05:00,10.890000,WAVS


In [41]:
annual_returns['date'] = pd.to_datetime(annual_returns['date']).dt.year
daily_returns['date'] = pd.to_datetime(daily_returns['date'], utc=True)
daily_returns['date'] = daily_returns['date'].dt.year
daily_returns


,date,close,ticker
0,1994,0.248125,AAPL
1,1994,0.235212,AAPL
2,1994,0.225065,AAPL
3,1994,0.225988,AAPL
4,1994,0.228755,AAPL
...,...,...,...
1315928,2024,10.840000,WAVS
1315929,2024,10.860000,WAVS
1315930,2024,10.890000,WAVS
1315931,2024,10.890000,WAVS


In [42]:
opcash_to_capex = opcash_to_capex[opcash_to_capex.groupby('ticker')['ticker'].transform('count') >= 10]
opcash_to_capex

,date,Operating cash flow,ticker,capex,Operating cash flow to capex
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440
...,...,...,...,...,...
3844,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646
3845,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703
3846,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643
3847,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084


In [43]:
opcash_to_capex['rank'] = opcash_to_capex.groupby('date')['Operating cash flow to capex'].rank(ascending=False)
pd.reset_option('display.max_rows', None)
opcash_to_capex.sort_values(by='rank')

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank
2366,2006,-3.819970e+07,PLUS,-34505417.0,90.329029,1.0
3397,1998,6.283000e+06,RELL,-4116000.0,65.510107,1.0
1377,2009,2.617250e+08,AMKR,-218177000.0,83.361162,1.0
1370,2016,7.294020e+08,AMKR,-650038000.0,89.119306,1.0
2703,2002,4.953000e+06,SPNS,-3743000.0,75.570361,1.0
...,...,...,...,...,...,...
3658,2021,1.525200e+07,GSIT,-203000.0,1.330973,116.0
3117,2017,-5.223500e+07,CERS,-353000.0,0.675792,116.0
764,2021,1.203900e+09,CHKP,-15900000.0,1.320708,117.0
2845,2015,6.905500e+07,ERII,-572000.0,0.828325,117.0


In [44]:
opcash_to_capex['quintile'] = pd.qcut(opcash_to_capex['rank'], q=5, labels=False, duplicates='drop') + 1


In [45]:
quintile_portfolios = {}

for q in range(1, 6):
    quintile_portfolios[q] = opcash_to_capex[opcash_to_capex['quintile'] == q].copy()


In [59]:
opcash_to_capex[#(opcash_to_capex['date']==2010) &
                (opcash_to_capex['quintile']==3)].sort_values(by='rank')['Operating cash flow to capex'].std()

5.459908881667268

In [47]:
returns = opcash_to_capex.merge(annual_returns, how='inner', on=['ticker', 'date'])
avg_returns = returns.groupby(['date', 'quintile'])['close(%)'].mean()
# returns_roll_3 = returns.groupby(['quintile', 'date'])['close(%)'].rolling(window=3).mean().reset_index()
# returns_roll_3
returns

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank,quintile,close(%)
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789,34.0,2,0.353998
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199,81.0,4,-0.392748
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761,76.0,4,0.275682
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920,81.0,4,0.438959
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440,66.0,4,0.470177
...,...,...,...,...,...,...,...,...
1630,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646,45.0,3,0.079277
1631,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703,91.0,5,-0.480874
1632,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643,85.0,4,0.691813
1633,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084,93.0,5,0.486779


In [48]:
avg_returns = pd.DataFrame(avg_returns).groupby('quintile')['close(%)'].mean()
avg_returns

quintile
1   -0.464471
2    0.002483
3    0.099159
4    0.089489
5    0.002712
Name: close(%), dtype: float64

In [49]:
excess_returns = pd.DataFrame(avg_returns-returns['close(%)'].mean()).rename(columns={'close(%)':'excess returns'})
# excess_returns.merge(avg_returns, on='quintile').to_csv('excess_returns.csv')
excess_returns

,excess returns
quintile,
1,-0.460741
2,0.006213
3,0.102889
4,0.093219
5,0.006442


In [52]:
avg_returns_year = returns.groupby(['date', 'quintile'])['close(%)'].mean()
# pd.reset_option('display.max_rows',)
pd.DataFrame(avg_returns_year).to_csv('avg_returns_year.csv')

In [14]:
returns['close(%)'].mean()

-0.003730182638564316